In [8]:
#Скрипт выгрузки из таблицы бд в csv файл с логированием
import pandas as pd
from sqlalchemy import create_engine
import time

# Параметры подключения к базе данных
db_user = 'postgres'
db_password = '1503'
db_host = 'localhost'
db_port = '5432'
db_name = 'project'

# Строка подключения к базе данных
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

# Определение переменной table_name
table_name = 'dm_f101_round_f'

# Функция отправки переменных с информацией в бд логов
def log_to_database(engine, log_info):
    table_or_function_name = log_info['table_or_function_name']
    ready_to_start = log_info['ready_to_start']
    start_time = log_info['start_time']
    end_time = log_info['end_time']
    count_record = log_info['count_record']

    query = """
        INSERT INTO logs.log_info (table_or_function_name, ready_to_start, start_time, end_time, count_record)
        VALUES (%s, %s, %s, %s, %s)
    """

    try:
        with engine.connect() as conn:
            conn.execute(query, (table_or_function_name, ready_to_start, start_time, end_time, count_record))
        print('Данные логов успешно добавлены в базу данных', end='\n\n')
    except Exception as e:
        print(f'Ошибка загрузки логов в базу данных: {str(e)}')

# Запрос для подсчета строк в исходной таблице баз данных
with engine.connect() as conn:
    result = conn.execute(f'SELECT COUNT(*) FROM DM.DM_F101_ROUND_F')
    row_count = result.scalar()

# Запись о готовности начать работать
ready_to_start = time.strftime('%Y-%m-%d %H:%M:%S')
    
# Тайм-аут 5 секунд
time.sleep(5)
    
# Запись о начале работы с таблицей
start_time = time.strftime('%Y-%m-%d %H:%M:%S')

# Запрос к базе данных и сохранение результата в DataFrame
df = pd.read_sql_query('SELECT * FROM DM.DM_F101_ROUND_F', engine)

# Путь для сохранения файла CSV на рабочий стол
file_path = '/Users/nikitos/Desktop/dm.dm_f101_round_f.csv'

# Сохранение данных из DataFrame в файл CSV с разделителем ":" и кодировкой UTF-8
df.to_csv(file_path, index=False, sep=';', encoding='utf-8')

# Фиксация времени окончания скрипта
end_time = time.strftime('%Y-%m-%d %H:%M:%S')


# Фиксация времени окончания скрипта
end_time = time.strftime('%Y-%m-%d %H:%M:%S')

# Запись в переменную лога при завершении работы с таблицей
log_info = {
            'ready_to_start': ready_to_start,
            'table_or_function_name': table_name,
            'start_time': start_time,
            'end_time': end_time,
            'count_record': row_count
            }
log_to_database(engine, log_info)

print(f'Данные таблицы успешно выгружены в файл {file_path}')


Данные логов успешно добавлены в базу данных

Данные таблицы успешно выгружены в файл /Users/nikitos/Desktop/dm.dm_f101_round_f.csv


In [7]:
#Скрипт загрузки csv
import pandas as pd
from sqlalchemy import create_engine
import chardet

db_user = 'postgres'
db_password = '1503'
db_host = 'localhost'
db_port = '5432'
db_name = 'project'

connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

csv_file_path = '/Users/nikitos/Desktop/dm.dm_f101_round_f_v2.csv'
F101_ROUND_F_v2_df = pd.read_csv(csv_file_path, sep=';')

F101_ROUND_F_v2_df.to_sql('dm_f101_round_f_v2', engine, schema='dm', if_exists='replace', index=False)

18